In [10]:
import logging
import json
from pathlib import Path
import os

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
                    level=logging.WARNING)

PROJECT_ROOT = Path(os.getcwd()).parents[1].resolve()
DATA_PATH = os.path.join(PROJECT_ROOT, 'data')

MODEL_FILE = 'Model_DomesticDeclarations'

# load json from file
with open(os.path.join(DATA_PATH, MODEL_FILE + '.json'), 'r') as json_file:
    model_json = json.loads(json_file.read())

In [11]:
import pandas as pd
from semconstmining.parsing.parser import get_elements_flat
from semconstmining.parsing.model_to_log import check_soundness, create_log
from semconstmining.parsing.conversion.bpmnjsonanalyzer import fromJSON
from semconstmining.parsing.conversion.jsontopetrinetconverter import JsonToPetriNetConverter

f, l, _ = fromJSON(model_json)
elements = pd.DataFrame.from_records(get_elements_flat(model_json, f, l)).set_index('eid')
elements

,category,label
eid,,
Flow_02aimhs,SequenceFlow,
Flow_0fj368l,SequenceFlow,
Flow_188govb,SequenceFlow,
Flow_0t3vcdm,SequenceFlow,
Flow_0pa3yag,SequenceFlow,
Flow_0wu8i49,SequenceFlow,
Flow_10cx5vi,SequenceFlow,
Flow_0vaoqjk,SequenceFlow,
Flow_1brr09y,SequenceFlow,


In [12]:
converter = JsonToPetriNetConverter()

net, im, fm = converter.convert_from_parsed(f, l)
is_sound = True #check_soundness(net, im, fm)
print(is_sound)
if is_sound:
    log = create_log(net, im, fm, elements)
    print(log)

True
[{'attributes': {}, 'events': [{'concept:name': 'Declaration SUBMITTED by EMPLOYEE', 'time:timestamp': datetime.datetime(1970, 4, 26, 18, 46, 41), 'eid': 'Activity_0y75frc', 'category': 'Task'}, '..', {'concept:name': 'Declaration REJECTED by EMPLOYEE', 'time:timestamp': datetime.datetime(1970, 4, 26, 18, 46, 48), 'eid': 'Activity_1ua672n', 'category': 'Task'}]}, '....', {'attributes': {}, 'events': [{'concept:name': 'Declaration SUBMITTED by EMPLOYEE', 'time:timestamp': datetime.datetime(1970, 4, 26, 18, 51, 6), 'eid': 'Activity_0y75frc', 'category': 'Task'}, '..', {'concept:name': 'Declaration REJECTED by EMPLOYEE', 'time:timestamp': datetime.datetime(1970, 4, 26, 18, 51, 14), 'eid': 'Activity_1ua672n', 'category': 'Task'}]}]


In [13]:
from semconstmining.declare.enums import Template
from semconstmining.declare.declare import Declare


d4py = Declare()
d4py.config.CONSTRAINT_TYPES_TO_IGNORE = [Template.CHAIN_RESPONSE.templ_str,
                                           Template.CHAIN_PRECEDENCE.templ_str, Template.CHAIN_SUCCESSION.templ_str,
                                           Template.CHOICE.templ_str, Template.NOT_CHAIN_RESPONSE.templ_str,
                                             Template.NOT_CHAIN_PRECEDENCE.templ_str]
res = set()
for i, trace in enumerate(log):
    trace.attributes["concept:name"] = str(i)
d4py.log = log
d4py.compute_frequent_itemsets(min_support=0.0, len_itemset=2, algorithm="apriori")
d4py.discovery(consider_vacuity=True, max_declare_cardinality=2, plain=True)
individual_res, associations = d4py.filter_discovery(min_support=0.99, plain=True)
res.update(const for const, checker_results in individual_res.items() if "[]" not in const
           and "[none]" not in const)
res

{'Absence2[Declaration APPROVED by ADMINISTRATION] | |',
 'Absence2[Declaration APPROVED by BUDGET OWNER] | |',
 'Absence2[Declaration FINAL_APPROVED by SUPERVISOR] | |',
 'Absence2[Declaration REJECTED by ADMINISTRATION] | |',
 'Absence2[Declaration REJECTED by BUDGET OWNER] | |',
 'Absence2[Declaration REJECTED by EMPLOYEE] | |',
 'Absence2[Declaration REJECTED by SUPERVISOR] | |',
 'Absence2[Declaration SUBMITTED by EMPLOYEE] | |',
 'Absence2[Payment Handled] | |',
 'Absence2[Request Payment] | |',
 'Alternate Precedence[Declaration APPROVED by ADMINISTRATION, Declaration APPROVED by BUDGET OWNER] | | |',
 'Alternate Precedence[Declaration APPROVED by ADMINISTRATION, Declaration FINAL_APPROVED by SUPERVISOR] | | |',
 'Alternate Precedence[Declaration APPROVED by ADMINISTRATION, Declaration REJECTED by BUDGET OWNER] | | |',
 'Alternate Precedence[Declaration APPROVED by ADMINISTRATION, Declaration REJECTED by SUPERVISOR] | | |',
 'Alternate Precedence[Declaration APPROVED by ADMINIST

In [14]:
records = []
# create records for a data frame with columns type, activities, logging_error, compliance_error, affects_outcome, affects_performance, exception. type is the template string, e.g., Precedence, activities are the activities of the template, and the others are just 0.
for const in res:
    template = const.split("[")[0]
    activities = const.split("[")[1].split("]")[0]
    records.append({"type": template, "activities": activities, "logging_error": 0, "compliance_error": 0, "affects_outcome": 0, "affects_performance": 0, "exception": 0})
df = pd.DataFrame.from_records(records)
df

,type,activities,logging_error,compliance_error,affects_outcome,affects_performance,exception
0,Not Precedence,"Declaration REJECTED by EMPLOYEE, Declaration ...",0,0,0,0,0
1,Not Responded Existence,"Declaration REJECTED by EMPLOYEE, Request Payment",0,0,0,0,0
2,Alternate Precedence,"Declaration SUBMITTED by EMPLOYEE, Declaration...",0,0,0,0,0
3,Not Response,"Request Payment, Declaration SUBMITTED by EMPL...",0,0,0,0,0
4,Responded Existence,"Declaration APPROVED by BUDGET OWNER, Declarat...",0,0,0,0,0
...,...,...,...,...,...,...,...
256,Precedence,"Declaration APPROVED by ADMINISTRATION, Declar...",0,0,0,0,0
257,Not Response,"Request Payment, Declaration REJECTED by SUPER...",0,0,0,0,0
258,Not Precedence,"Declaration APPROVED by ADMINISTRATION, Declar...",0,0,0,0,0
259,Not Response,"Declaration REJECTED by ADMINISTRATION, Reques...",0,0,0,0,0


In [15]:
# export the data frame to a csv file
df.to_csv(os.path.join(DATA_PATH, MODEL_FILE + '_constraints.csv'), index=False)